# Plot Cote D'Ivoire Monthly Rainfall Data

This will show several plotting examples with comments.

In [ ]:
import glob
import numpy as np
import xarray as xr
import hvplot.xarray
import hvplot.pandas
#import geoviews.feature as gf
import panel as pn

#set the backend and the renderer
import holoviews as hv
hv.extension('bokeh')
br = hv.renderer('bokeh')

In [ ]:
# Set the data directory 
nc_dir = '../data/rainfall/'

## Load Data into Xarray Dataset

Load multiple netCDF-4 files using Xarray. Call compute() to load the data into memory since xr.open_mfdataset() uses Dask to lazily load the files resulting in chunked data. Calling compute() to put all the data into memory.  

In [ ]:
# 1. Load the netCDF-4 file into an Xarray dataset
# 2. Print the Dataset header
dpr_all_ds = xr.open_mfdataset(nc_dir + '*.nc4').compute()
dpr_all_ds

In [ ]:
# 1. Convert the Xarray Dataset (gridded data) to a Pandas Dataframe (tabular data)
#    reset_index() causes the Dataframe's first column to be numbers instead of lon/lat/time
# 2. Print the Dataframe header
dpr_all_df = dpr_all_ds.to_dataframe().reset_index()
dpr_all_df

## Bar plots

In [ ]:
# Monthly Mean Rainfall
# 1. drop() the Dataframe columns we don't need
# 2. Group the rows by month
# 3. Calculate the mean() of each month
# 4. Rename the columns to something more appropriate for the plot labels
# 5. Call hvplot.bar(), the Hvplot barplot function.
# 6. opts(disable...) to remove the Bokeh toolbar from the plot

dpr_all_df.\
    drop(columns=['lat','lon','all_rainfall']).\
    groupby(by=dpr_all_df.time.dt.month).\
    mean().\
    rename(columns={'conv_non_sh_rainfall': 'Deep Conv.', 'conv_sh_rainfall': 'Shallow Conv.','sf_rainfall': 'Stratiform'}).\
    hvplot.bar(rot=90, title='West Africa Monthly Mean Rainfall by Rain Type, 2015-2021', frame_height=200, frame_width=800, fontscale=1.5, 
               xlabel='Month, Rain Type', ylabel='Rainfall (mm/day)').opts(toolbar='disable')

In [ ]:
# Same as above, but calculate the annual mean in the groupby function

dpr_all_df.\
    drop(columns=['lat','lon','all_rainfall']).\
    groupby(by=dpr_all_df.time.dt.year).\
    mean().\
    rename(columns={'conv_non_sh_rainfall': 'Deep Conv.', 'conv_sh_rainfall': 'Shallow Conv.','sf_rainfall': 'Stratiform'}).\
    hvplot.bar(rot=80, title='West Africa Annual Mean Rainfall by Type', frame_height=200, frame_width=800, fontscale=1.5).opts(toolbar='disable')

## Violin plots

In [ ]:
# Create 2 violin plots of monthly rainfall distributions for stratiform and deep convective rainfall

# Stratiform Violin
sf_violin_plot = (dpr_all_df[dpr_all_df['sf_rainfall'] >= 0.1]).sort_values(by='time').\
                     hvplot.violin(y='sf_rainfall', by='time.month', title='West Africa Stratiform Rainfall', ylim=(0,20), frame_width=1000, grid=True,
                                  xlabel='Month', ylabel='Rainfall (mm/day)')
# Stratiform Median Line
sf_line_plot = (dpr_all_df[dpr_all_df['sf_rainfall'] >= 0.1]).sort_values(by='time').groupby(by=dpr_all_df.time.dt.month).median().\
                    hvplot.line(y='sf_rainfall', line_color='red')

# Overlay the median line plot on the violin plot
sf_plot = sf_violin_plot * sf_line_plot


# Deep Convective Violin
conv_deep_violin_plot = (dpr_all_df[dpr_all_df['conv_non_sh_rainfall'] >= 0.1]).sort_values(by='time').\
                             hvplot.violin(y='conv_non_sh_rainfall', by='time.month', title='West Africa Stratiform Rainfall', ylim=(0,20), frame_width=1000, grid=True,
                                           xlabel='Month', ylabel='Rainfall (mm/day)')
# Deep Convective Median Line
conv_deep_line_plot = (dpr_all_df[dpr_all_df['conv_non_sh_rainfall'] >= 0.1]).sort_values(by='time').groupby(by=dpr_all_df.time.dt.month).median().\
                            hvplot.line(y='conv_non_sh_rainfall', line_color='red')

# Overlay the median line plot on the violin plot
conv_deep_plot = conv_deep_violin_plot * conv_deep_line_plot

# Show both stratiform and convective plots in a single column
(sf_plot + conv_deep_plot).cols(1)